Basic things:

In [4]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import pandas as pd
from IPython.display import display, HTML

Here we create Downloader:

In [11]:
class Downloader:
    def __init__(self, link):
        self.link = link
        
    def download(self):
        # This will download page html
        uClient = uReq(self.link)
        page_html = uClient.read()
        uClient.close()
        return soup(page_html, "lxml")
    
    def get_titles(self):
        html = self.download()
        td = html.findAll('span', {"class":"title"})
        titles = []
        for ind in td:
            # lstrip and rstrip remove symbols from sides, strip removes white spaces
            titles.append(str(ind).lstrip('<span class="title">').rstrip('span>').rstrip('</'))
        return titles
    
    def get_release_dates(self):
        html = self.download()
        td = html.findAll('div', {"class":"col search_released responsive_secondrow"})
        release_dates = []
        for ind in td:
            # lstrip and rstrip remove symbols from sides, strip removes white spaces
            release_dates.append(str(ind).rstrip('</div>').split(">")[-1])
        return release_dates
    
    def reviews(self):
        html = self.download()
        td = html.findAll('span', {"class":"search_review_summary positive"})
        reviews = []
        for ind in td:
            # lstrip and rstrip remove symbols from sides, strip removes white spaces
            reviews.append(str(ind).split("html=")[-1])
        return reviews
     
    def get_share_positive_reviews(self):
        text = self.reviews()
        shares = []
        for percent in text:
            shares.append(percent.split("%")[0].split("br&gt;")[1])
        return shares
        
    def get_number_user_reviews(self):  
        text = self.reviews()
        numbers = []
        for number in text:
            start = number.find("of the ") + len("of the ")
            end = number.find(" user reviews")
            numbers.append(number[start:end].replace(","," "))
        return numbers
        
    def get_prices(self):
        html = self.download()
        td = html.findAll('div', {"class":"col search_price_discount_combined responsive_secondrow"} or 
                          {"class":"col search_price discounted responsive_secondrow"})
        prices = []
        for ind in td:
            if "888888" not in str(ind):
                prices.append(str(ind).split("\r\n")[-1].split("</div>\n</div>")[0].strip())
            else:
                start1 = str(ind).find("><strike>") + len("><strike>")
                end1 = str(ind).find("</strike>")
                prices.append(str(ind)[start1:end1])
        return prices
    
    def get_price_after_sale(self):
        html = self.download()
        td = html.findAll('div', {"class":"col search_price_discount_combined responsive_secondrow"} or 
                          {"class":"col search_price discounted responsive_secondrow"})
        sales = []
        for ind in td:
            if "888888" not in str(ind):
                sales.append("NA")
            else:
                sales.append(str(ind).split("br/>")[-1].split("€")[0]+"€")
        return sales
    
    def get_rate_of_sale(self):
        html = self.download()
        td = html.findAll('div', {"class":"col search_price_discount_combined responsive_secondrow"} or 
                          {"class":"col search_price discounted responsive_secondrow"})
        percent = []
        for ind in td:
            if "888888" not in str(ind):
                percent.append("NA")
            else:
                start = str(ind).find(">\n<span>-")+len(">\n<span>-")
                end = str(ind).find("%")
                percent.append(str(ind)[start:end]+"%")
        return percent
    
    def dataf(self):
        titles = self.get_titles()
        dates = self.get_release_dates()
        share_reviews = self.get_share_positive_reviews()
        number_reviews = self.get_number_user_reviews()
        normal_prices = self.get_prices()
        sale_price = self.get_price_after_sale()
        sale_rate = self.get_rate_of_sale()
        
        
        self.data = pd.DataFrame({
             'title': pd.Series(titles),
             'release date': pd.Series(dates),
             'share of positive reviews (in %)': pd.Series(share_reviews),
             'total number of reviews': pd.Series(number_reviews),
             'normal price': pd.Series(normal_prices),
             'sale price if there is a sale': pd.Series(sale_price),
             'sale rate (in %)': pd.Series(sale_rate)})
        return self.data
   
    def hoarder(self):
        urls = []
        for i in range(3):
            urls.append(self.link + f"&page={1+i}")
        return urls
    
    def download_data(self):
        urls = self.hoarder()
        Frame = pd.DataFrame()
        for url in urls:
            Frame = Frame.append(pd.DataFrame(data = Downloader(url).dataf()))
        Frame.index = range(3*25)
        return Frame

In this cell we check input the link and check html:

In [13]:
first = Downloader('https://store.steampowered.com/search/?sort_by=Reviews_DESC&os=win&filter=globaltopsellers')
#first.download() # in order to explore page html in a reasonable way, one can use online javascript beautifier, available at:
# beautifier.io
dataset = first.download_data()
dataset

,title,release date,share of positive reviews (in %),total number of reviews,normal price,sale price if there is a sale,sale rate (in %)
0,The Witcher 3: Wild Hunt - Expansion Pass,"19 May, 2015",99,3 338,"24,99€",NA,NA
1,Senren＊Banka,"14 Feb, 2020",99,2 688,"29,99€","26,99€",10%
2,Aseprite,"22 Feb, 2016",99,2 649,"14,99€",NA,NA
3,A Short Hike,"30 Jul, 2019",99,2 584,"6,59€","4,94€",25%
4,Doki Doki Literature Club Fan Pack,"22 Sep, 2017",99,1 490,"9,99€",NA,NA
...,...,...,...,...,...,...,...
70,RESIDENT EVIL 2 / BIOHAZARD RE:2,"24 Jan, 2019",97,42 210,"39,99€",NA,NA
71,BattleBlock Theater®,"15 May, 2014",97,41 152,"14,99€",NA,NA
72,Mirror,"19 Apr, 2018",97,38 286,"1,99€",NA,NA
73,Slime Rancher,"1 Aug, 2017",97,35 340,"19,99€",NA,NA


Here we check single attribute, if and how they work:

In [82]:
first.get_titles()
first.get_release_dates()
first.reviews()
first.get_share_positive_reviews()
first.get_number_user_reviews()
first.get_prices()
first.get_price_after_sale()
first.get_rate_of_sale()

['NA',
 '10%',
 'NA',
 '25%',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 '20%',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 '22%',
 '25%',
 '25%']

Now let's check the dataframe:

In [232]:
first.dataf()

,title,release date,share of positive reviews (in %),total number of reviews,normal price,sale price if there is a sale,sale rate (in %)
0,The Witcher 3: Wild Hunt - Expansion Pass,"19 May, 2015",99,3 338,"24,99€",NA,NA
1,Senren＊Banka,"14 Feb, 2020",99,2 673,"29,99€","26,99€",10%
2,Aseprite,"22 Feb, 2016",99,2 639,"14,99€",NA,NA
3,A Short Hike,"30 Jul, 2019",99,2 556,"6,59€","4,94€",25%
4,Doki Doki Literature Club Fan Pack,"22 Sep, 2017",99,1 490,"9,99€",NA,NA
5,rian HD,"8 May, 2017",99,1 209,"9,99€",NA,NA
6,This War of Mine: War Child Charity,"9 Mar, 2015",99,1 180,"0,99€",NA,NA
7,The Scriptwelder Collection,"<div class=""col search_released responsive_sec...",99,652,"19,52€",NA,NA
8,Don't Escape: 4 Days to Survive,"11 Mar, 2019",99,585,"14,99€",NA,NA
9,Smile For Me - Mouthful Edition,"<div class=""col search_released responsive_sec...",99,511,"14,--€",NA,NA
